importowanie bibliotek

In [1]:

import numpy as np
import pytest
from sklearn.linear_model import Ridge

$ \frac{1}{2}\sum_{i=1}^{n}{(y_i-\hat{f}{(x_i)})}^2 $ \
Gdzie to jest równe \
$ \frac{1}{2}\sum_{i=1}^{n}{(y_i-{\beta}_0-{{\beta}_1}x_{i1}-{{\beta}_2}x_{i2}-...-{{\beta}_n}x_{in}) }^2 $ \
ale u nas 
$ \frac{1}{2}\sum_{i=1}^{n}{(y_i-{{\beta}_0}-{{\beta}_1}x_{i1}-{{\beta}_2}x_{i2}-...-{{\beta}_m}x_{im}) }^2 + {\frac{1}{2}}{\alpha}\sum_{j=1}^{m}{({{\beta}_j}^2)} $ \
Cel: Znaleźć ${\beta}_0,{\beta}_1 ...itd$ dla których $L(\beta)$ jest minimalna

1. Wybieramy jakieś początkowe $\beta = {\beta}_0,\cdots,{\beta}_n$ \
2. Powtarzamy \
    Liczymy $\frac{dL}{d{\beta}_j}$ używając definicji ${beta}_j$ \
    Bierzemy (nowe ${\beta}_j$ = (stare ${\beta}_j$) - $c\frac{dL}{d{\beta}_j} $)  
    gdzie c to pewna stała dodatnia (stała uczenia np: 0.01)

In [ ]:
class RidgeRegr:
    def __init__(self, alpha=0.0, tol=1e-12):
        self.alpha = float(alpha)
        self.tol = float(tol)
        self.beta = None 

    def dL(self, X, Y,j):
        """Compute the derivate of the loss function with respect to beta_j

        Args:
            X (np.ndarray): x values
            Y (np.ndarray): y values
            j (int): Index of the coefficient

        Returns:
            float: The derivative of the loss function with respect to beta_j
        """

        n, m = X.shape
        suma = 0

        for i in range(n):
            suma += (Y[i] - X[i,:] @ self.beta) *( -X[i,j])
        return suma + (self.alpha * self.beta[j] if j > 0 else 0.0)
         

    def fit(self, X, Y, c=0.01):
        """Fits the model to the data using gradient descent

        Args:
            X (np.ndarray): x values
            Y (np.ndarray): y values
            c (float, optional): Learning rate. Defaults to 0.0001.
        
        Returns:
            RidgeRegr: The fitted model
        """
        
        n, m = X.shape

        X = np.hstack((np.ones((X.shape[0],1)),X))

        self.beta = np.zeros(m + 1, dtype=float)

        while True:

            beta_old = self.beta.copy()

            for j in range(m+1):
                self.beta[j] -= c * self.dL(X, Y, j)

            if np.linalg.norm(self.beta -  beta_old) < self.tol:
                break

        return self

    def predict(self, X):
        """Predicts the output for the given input data

        Args:
            X (np.ndarray): x values

        Returns:
            np.ndarray: Predicted y values"""

        n, m = X.shape
        X = np.hstack((np.ones((X.shape[0],1)),X))

        return X @ self.beta

In [3]:
def test_RidgeRegressionInOneDim():
    X = np.array([1,3,2,5]).reshape((4,1))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,2,10]).reshape((3,1))
    alpha = 0.3
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    print(expected)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    print(actual)
    assert list(actual) == pytest.approx(list(expected), rel=1e-5)

def test_RidgeRegressionInThreeDim():
    X = np.array([1,2,3,5,4,5,4,3,3,3,2,5]).reshape((4,3))
    Y = np.array([2,5, 3, 8])
    X_test = np.array([1,0,0, 0,1,0, 0,0,1, 2,5,7, -2,0,3]).reshape((5,3))
    alpha = 0.4
    expected = Ridge(alpha).fit(X, Y).predict(X_test)
    print(expected)
    actual = RidgeRegr(alpha).fit(X, Y).predict(X_test)
    print(actual)
    assert list(actual) == pytest.approx(list(expected), rel=1e-3)

testy

In [5]:
test_RidgeRegressionInOneDim()
test_RidgeRegressionInThreeDim()

[ 1.88950276  3.38121547 15.31491713]
[ 1.88950277  3.38121547 15.31491711]
[ 0.54685378 -1.76188321  1.58691716  5.15527388  3.66704391]
[ 0.54685371 -1.76188325  1.5869171   5.15527395  3.66704389]
